# Introduction

This notebooks provides the minimum necessary code to access data from egauge's API, the software currently used in both DHHL and the Frogs Project. For an overview of the architecture, see the readme.md file of the project where is notebook is hosted.



## 1. Required Input Parameters

### 1.1 Host

Currently eGauge API is [hosted by the eGauge company](https://www.egauge.net/eguard/). Contrary to the usual setup where the data resides in a database and the API exposes access to it in a server, the eGauge API does **not** store any eGauge data. Instead, the data is stored in the sensor itself. The egaugeAPI serves only as a in between layer between this Python Notebook and the sensor, in order to abstract how to communicate with the sensor directly. Because of that, **the sensors must be connected to the internet, regardless of where they are deployed, in order for the data to be available via API at any point in time**. 

While the eGauge website requires authentication to visualize the sensor IDs (required to request data from the API) and their physical location, once the sensor ID is known the access to the data itself does not require authentication. As such, provided the sensor ID is known, any user in the world can request data from the sensor via the API.

The API website where sensor IDs is are available is shown below for reproducibility purposes:


![egauge_portfolio](img/egauge_portfolio.png)

As it can be observed, on the **Status** and **Available** columns, because the data is available only in the sensor, requesting data from a particular sensor may require several attempts if the requests are automated. Accordingly, the specified time range upon every request must be updated over every failed attempt. 

Stored data is updated once a minute, and it can be fetched using the following host template [1]:

``` 
http://egauge{}.egaug.es/cgi-bin/egauge-show?
```

In [1]:
host = 'http://egauge{}.egaug.es/cgi-bin/egauge-show?'

### 1.2 Sensor 

You should contact a member of ERDL if you are interested in data by any specific sensor managed by our group. In this example notebook, we will use the eGauge whose sensor ID `31871`. 

In [2]:
sensor_id = 31871

The list of available parameters [1] at the time this Python notebook was made is shown below for reproducibility. 

![eguard_api_parameters](img/egauge_parameters.png)

We only use 5 of the ones available above.

### 1.3 Start and End Date

The start and end date are specified by parameters **t** (14th row top-bottom) and **f** (13th row top-bottom) respectively, and must be specified in [unix timestamp](https://en.wikipedia.org/wiki/Unix_time).  

The following function converts a timestamp in the human readable format to unix timestamp by using the following format: `<date>T<time><timezone>`, where:

 * `<date>`: YYYY-MM-DD
 * `<time>`: hh:mm:ss
 * `<timezone>`: (-)zz
 
Hawaii Timezone is -10. 

In [3]:
import arrow
iso_start_datetimez = '2017-09-13T15:19:00-10'
iso_end_datetimez = '2017-09-13T15:21:00-10'

start_timestamp = arrow.get(iso_start_datetimez).timestamp
end_timestamp = arrow.get(iso_end_datetimez).timestamp

### 1.3 Time Resolution 

The time resolution, as specified by parameter **m** (6th row top-bottom), specifies the output resolution will be in **minutes**. 

In [4]:
minutes = 'm'

### 1.4 Export Format 

The export format, as specified by parameter **c** (4th row top-bottom), specifies the output will be in **CSV**.

In [5]:
output_csv = 'c'

### 1.5 Compression

The compression method, as specified by parameter `C` (10th row top-bottom), specifies the use of **delta-compression**.

In [6]:
delta_compression = 'C'

## Request Readings 

In [7]:
import requests

host = host.format(sensor_id) + '&' + minutes + '&' + output_csv + '&' + delta_compression 
time_window = {'t': start_timestamp, 'f': end_timestamp}

print(host)

http://egauge31871.egaug.es/cgi-bin/egauge-show?&m&c&C


In [8]:
r = requests.get(host,params=time_window)
print(r)

<Response [200]>


The returned .csv will be in the format of a string, so we interpret it accordingly.

In [24]:
string_csv_file = r.text    

rows = [[y for y in x.split(',')] for x in string_csv_file.splitlines()]
headers = [h.replace('"','') for h in rows.pop(0)]
columns = list(zip(*rows))
print({h: list(map(float,columns[i])) for i,h in enumerate(headers)})

{'Date & Time': [1505352000.0, 1505351940.0], 'Usage [kW]': [57.796533333, 57.3584], 'Generation [kW]': [-0.0, -0.0], 'Phase A+ [kW]': [17.0116, 16.721866667], 'Phase B+ [kW]': [21.236183333, 21.0492], 'Phase C+ [kW]': [19.54875, 19.587333333], 'Phase A voltage [V]': [282.492763418, 282.899163437], 'Phase B voltage [V]': [282.790113432, 283.196396784], 'Phase C voltage [V]': [283.992580156, 284.325363505]}


# 4. References

 * [1] See [eGauge XML API](https://www.egauge.net/docs/egauge-xml-api.pdf) for a exaustive list of available parameters for the eGauge API. Section 3, stored data, was used to create this Python Notebook.